In [2]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 1.9 MB/s eta 0:00:01     |██████████████████████████████▎ | 1.4 MB 1.9 MB/s eta 0:00:01


In [5]:
cd

/opt/ml


In [20]:
from difflib import SequenceMatcher
import json
from tqdm.notebook import tqdm
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4

def similar2(ontology, preds, only_bleu=False, with_bleu=False):
    sim = 0
    bleu_score = 0
    splited_preds = preds.split()
    preds_length = len(splited_preds)
    for pred in preds.split():
        #split 후 평균
        sim += SequenceMatcher(None, ontology, pred).ratio()
        bleu_score += bleu(ontology, pred)
    sim /= preds_length
    bleu_score /= bleu_score
    if not only_bleu and not with_bleu:
        return sim
    if only_bleu:
        return bleu_score
    return (sim + bleu_score) / 2

def similar(ontology, preds, only_bleu=False, with_bleu=False):
    sim = SequenceMatcher(None, ontology, preds).ratio()
    if not only_bleu and not with_bleu:
        return sim
    bleu_score = bleu(ontology, preds)
    if only_bleu:
        return bleu_score
    return (sim + bleu_score) / 2
num = 48
with open(f"/opt/ml/predictions/post-cleaning-{num}.csv", "rb") as f:
    a = json.load(f)
cnt = 0
all_cnt = 0
for key, values in a.items():
    for i, value in enumerate(values):
        if "시간" in value:
            continue
        if ":" in value or "=" in value or "&" in value or "(" in value or ")" in value :
            print(f" before : {value}", end="\t")

        if "=" in value:
            a[key][i] = value.replace(" = ", "=")

        if "&" in value:
            a[key][i] = value.replace(" & ", "&")
        if "(" in value:
            value = value.replace("( ", "(")
            value = value.replace(" ) ", ")")
            value = value.replace(" )", ")")
            if value[-1] == "역":
                value = value.replace(" (", "(")
            a[key][i] = value
        if "동대문" in value and value[-1] == "(":
            value = "관광-이름-동대문 (흥인지문)"
            a[key][i] = value
        if "월드컵 경기장(" in value:
            a[key][i] = "관광-이름-서울 월드컵 경기장 (상암 월드컵 경기장)"
            
        if ":" in value or "=" in value or "&" in value or "(" in value or ")" in value:
            print(f" after : {a[key][i]}")
            cnt+=1
        
        all_cnt+=1
print(f"{cnt} / {all_cnt}")

predctions = a

with open("/opt/ml/input/data/train_dataset/ontology.json", "rb") as f:
    ontology = json.load(f)

all_values = sorted(set(sum(ontology.values(), [])))
name_values = sorted(set(ontology['관광-이름'] + ontology['식당-이름'] + ontology['숙소-이름'] + ontology['택시-도착지'] + ontology['택시-출발지']))
a = ontology['관광-이름'].copy()
b = ontology['숙소-이름'].copy()
c = ontology['식당-이름'].copy()
ontology['관광-이름'] = [n for n in name_values if n not in b + c + ontology['지하철-도착지']]
ontology['숙소-이름'] = [n for n in name_values if n not in a + c + ontology['지하철-도착지']]
ontology['식당-이름'] = [n for n in name_values if n not in a + b + ontology['지하철-도착지']]


cnt = 0
for guid, states in tqdm(predctions.items()):
    flag = False
    for i, state in enumerate(states):
        dom, slot, value = state.split('-')
        domslot = f"{dom}-{slot}"
        if "시간" in slot:
            continue
        if value in all_values:
            continue
        if ('게스트' in value or '호텔' in value or '모텔' in value or '하우스' in value) and '그린' not in value:
            value = value.split(' ')[0]
        if domslot in ['택시-출발지', '택시-도착지']:
            values = all_values
        else:
            values = ontology[domslot]
        #value 정답, 예측
        similarities = sorted([(v, similar(value, v, with_bleu=True)) for v in values] ,reverse=True, key=lambda x: x[1])
        print(domslot, value, "\t=>\t", similarities[:5])

        predctions[guid][i] = f"{domslot}-{similarities[0][0]}"
        flag = True
    if flag:
        cnt+=1
print(f"{cnt} / 14771")
json.dump(
    predctions,
    open(f"/opt/ml/predictions/final-cleaning-{num}.csv", "w"),
    indent=2,
    ensure_ascii=False,
)


 before : 택시-도착지-꿔=바로우	 after : 택시-도착지-꿔=바로우
 before : 택시-도착지-꿔=바로우	 after : 택시-도착지-꿔=바로우
 before : 택시-도착지-꿔=바로우	 after : 택시-도착지-꿔=바로우
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 택시-도착지-신촌(경의선)역	 after : 택시-도착지-신촌(경의선)역
 before : 관광-이름-롯데월드타워&몰	 after : 관광-이름-롯데월드타워&몰
 before : 관광-이름-롯데월드타워&몰	 after : 관광-이름-롯데월드타워&몰
 before : 택시-출발지-롯데월드타워&몰	 after : 택시-출발지-롯데월드타워&몰
 before : 관광-이름-롯데월드타워&몰	 after : 관광-이름-롯데월드타워&몰
 before : 택시-출발지-롯데월드타워&몰	 after : 택시-출발지-롯데월드타워&몰
 before : 관광-이름-롯데월드타워&몰	 after : 관광-이름-롯데월드타워&몰
 before : 택시-출발지-롯데월드타워&몰	 after : 택시-출발지-롯데월드타워&몰
 before : 식당-이름-꿔=바로우	 after : 식당-이름-꿔=바로우
 before : 식당-이름-꿔=바로우	 after : 식당-이름-꿔=바로우
 before : 택시-출발지-꿔=바로우

숙소-이름 그섬 	=>	 [('더 그린', 0.16666666666666666), ('그린하우스', 0.14285714285714285), ('그레이 호텔', 0.125), ('시그니처 서울', 0.1111111111111111), ('더 그레이 호텔', 0.1)]
숙소-이름 그섬 	=>	 [('더 그린', 0.16666666666666666), ('그린하우스', 0.14285714285714285), ('그레이 호텔', 0.125), ('시그니처 서울', 0.1111111111111111), ('더 그레이 호텔', 0.1)]
숙소-이름 그섬 	=>	 [('더 그린', 0.16666666666666666), ('그린하우스', 0.14285714285714285), ('그레이 호텔', 0.125), ('시그니처 서울', 0.1111111111111111), ('더 그레이 호텔', 0.1)]
택시-출발지 그섬 	=>	 [('더 그린', 0.16666666666666666), ('그린하우스', 0.14285714285714285), ('그레뱅뮤지엄', 0.125), ('그레이 호텔', 0.125), ('시그니처 서울', 0.1111111111111111)]
택시-출발지 그섬 	=>	 [('더 그린', 0.16666666666666666), ('그린하우스', 0.14285714285714285), ('그레뱅뮤지엄', 0.125), ('그레이 호텔', 0.125), ('시그니처 서울', 0.1111111111111111)]

5 / 14771
